In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
activity_ids = {
    0:"other (transient activities)",
    1 :"lying",
    2 :"sitting",
    3 :"standing",
    4 :"walking",
    5 :"running",
    6 :"cycling",
    7 :"Nordic walking",
    9 :"watching TV",
    10:"computer work",
    11:"car driving",
    12:"ascending stairs",
    13:"descending stairs",
    16:"vacuum cleaning",
    17:"ironing",
    18:"folding laundry",
    19:"house cleaning",
    20:"playing soccer",
    24:"rope jumping",
    }

In [3]:
def plot_activity(df_subject:pd.DataFrame, activity_id:int, t_window:float=60.):
    ax_cols = [
        ['IMU Hand Acc X1','IMU Hand Acc Y1', 'IMU Hand Acc Z1'],
        ['IMU Chest Acc X2','IMU Chest Acc Y2', 'IMU Chest Acc Z2'],
        ['IMU Anckle Acc X2','IMU Anckle Acc Y2', 'IMU Anckle Acc Z2']
        ]
    
    n_suplot = len(ax_cols)

    start = df_subject[df_subject["Activity ID"] == activity_id].index[0]

    plt.figure(figsize=(18, 10))
    plt.suptitle(activity_ids[activity_id])
    for i, cs in enumerate(ax_cols):
        ax = plt.subplot(n_suplot, 1, i+1)
        df_subject.loc[start: start+ t_window][cs].plot(ax=ax)
        df_subject.loc[start: start+ t_window]["Heart Rate"].plot(ax=ax)
        ax.grid(True)
        
    plt.tight_layout()
    plt.show()

In [4]:
def generate_cols():
    imu_cols= []
    cols = ["t_stamp", "Activity ID", "Heart Rate"]
    
    for i in ['Hand', "Chest", "Anckle"]:
        c = [
            f"IMU {i} Temperature", 
            f"IMU {i} Acc X1", f"IMU {i} Acc Y1", f"IMU {i} Acc Z1", 
            f"IMU {i} Acc X2", f"IMU {i} Acc Y2", f"IMU {i} Acc Z2", 
            f"IMU {i} Gyr X", f"IMU {i} Gyr Y", f"IMU {i} Gyr Z", 
            f"IMU {i} Mag X", f"IMU {i} Mag Y", f"IMU {i} Mag Z",
            f"invalid", f"invalid", f"invalid", "invalid"]
        imu_cols.extend(c)
        
    cols.extend(imu_cols)
    return cols

def load_subject(filename:str): 
    df_s = pd.read_csv(filename, delimiter=" ")
    df_s.columns = generate_cols()

    df_s = df_s.drop(columns=["invalid"])
    df_s = df_s.set_index(["t_stamp"])
    df_s["Heart Rate"] = df_s["Heart Rate"].ffill()
    
    return df_s

In [11]:
def create_activity_dataset(_dfs: list, selected_activity: int):
    datas = []
    for df in _dfs:
        activity = df[df["Activity ID"] == selected_activity]
        activity = activity.reset_index().drop(columns='t_stamp')
        
        datas.append(activity)
        
    activity_df = pd.concat(datas)
    
    return activity_df

In [14]:
SUBJECTS = [1, 5, 6, 8]
ACTIVITIES = [4, 5, 6, 24]

dfs = [load_subject(f'data/PAMP2/subject10{i}.dat') for i in SUBJECTS]

df_walking = create_activity_dataset(dfs, ACTIVITIES[0])
df_running = create_activity_dataset(dfs, ACTIVITIES[1])
df_cycling = create_activity_dataset(dfs, ACTIVITIES[2])
df_jumping = create_activity_dataset(dfs, ACTIVITIES[3])

In [16]:
SAVE_FOLDER = "data/PAMP2/per_activity"

df_walking.to_hdf(f"{SAVE_FOLDER}/walking.h5", key="data")
df_running.to_hdf(f"{SAVE_FOLDER}/running.h5", key="data")
df_cycling.to_hdf(f"{SAVE_FOLDER}/cycling.h5", key="data")
df_jumping.to_hdf(f"{SAVE_FOLDER}/jumping.h5", key="data")